# Estimación de media, total y proporción
    - Debido al teorema del límite central, es posible estimar la media de una manifestación aleatoria, se cual sea su función de probabilidad origina

    - Por la naturaleza estocástica de los fenómenos no es posible dar una estimación puntual de las características estadísticas, sino que la estimación se hace a través de intervalos de confianza

    - Se definen las siguientes medidas:
        - Media: sum(y_i)/N
        - Total: sum(y_i)
        - Proporción = sum(1 if condición(y_i) else 0) / N

### 1] Definir una población
    - Gracias al TLC, sin importar la distribución de la VA, es posible realizar las estimaciones de las medias, totales y proporciones

In [2]:
import pandas as pd
import numpy as np


# Define a population --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

# Settings
confidence = 0.90
N = 1000
n = 27
P = 0.15

# Define columns to be included
score = np.random.normal(loc=6.5, scale=1.5, size=N)
money = np.random.randint(low=200, high=1200, size=N)
is_male = np.where(np.random.uniform(size=N)<P, 1, 0)

# Put together data into a df
df_population = pd.DataFrame(np.vstack([score, money, is_male]).T, columns=["score", "money", "is_male"])



# Select a random sample --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

# Create auxiliary column to randomize the sample
df_population["mask"] = np.random.uniform(size=N)

# Sort and select elements randomly
df_sample = df_population.sort_values(by=["mask"])[:n]

# Drop auxiliary column
df_population.drop(["mask"], axis=1, inplace=True)

### 2] Estimador de la media

    - Los estimadores insesgados son:
            - mean = sum(y_i) / n
            - var = sum( (y_i-mean)^2 ) / (n-1)
            - V(mean) = (1 - n/N) * var / n
              donde a (1 - n/N) se le conoce como factor de corrección por finitud
            
    - Dichos estimadores tienen una varianza asociada

    - Las medias muestrales se distribuyen normalmente con los parámetros estimados previamente
            - Intervalo de confianza al alpha%:
            [ mean - var*normstdinv(alpha/2) , mean + var*normstdinv(alpha/2) ]
            
            - Sin embargo, como se utiliza var el estomador de V, en vez de emplear una distribución normal, se opta por utilizar T-Student con n-1 grados de libertad

In [3]:
# Analysis to be done with "score" variable

# Population parameters
p_mean = df_population["score"].mean()
p_var = df_population["score"].var(ddof=0)
print("Population | mean: "+str(round(p_mean,2))+" - var: "+str(round(p_var,2)))

# Estimation of parameters using the sample
mean = df_sample["score"].mean()
var = df_sample["score"].var(ddof=1)
print("Sample     | mean: "+str(round(mean,2))+" - var: "+str(round(var,2)))

# Calculate confidence interval
from scipy.stats import t
mean_var = (1 - n/N) * var /n
mean_std = mean_var**0.5
mean_interval =  mean + mean_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))
print("Confidence interval: ["+ str(round(mean_interval[0],2))+ ", "+ str(round(mean_interval[1],2))+"]")

# Accuracy of estimation
if p_mean > mean_interval[0] and p_mean < mean_interval[1]:
    print("\nValid!")
else:
    print("\nNot valid!")

Population | mean: 6.49 - var: 2.25
Sample     | mean: 6.39 - var: 1.79
Confidence interval: [5.96, 6.82]

Valid!


### 3] Estimador del total

    - Se define el total de una variable como la suma a través de todos los elementos
        total = sum(y_i)

    - Los estimadores insesgados son:
        - total = N * mean = sum( N/n * y_i)
        - V(total) = N^2 * (1-n/N) * S^2/n

    - El total de las muestras siguen una distribución normal con los parámetros descritos arriba

    - Para calcular los intervalos de confianza se sigue el procedimiento de la media.

In [419]:
# Analysis to be done with "money" variable

# Population paramateres
p_total = df_population["money"].sum()
print("Population | total: "+str(int(p_total)))

# Sample paramters
total = N/n * df_sample["money"].sum()
print("Sample     | total: "+str(int(total)))

# Calculate confidence intervals
total_var = N**2 * (1-n/N) * df_sample["money"].var(ddof=1)/n
total_std = total_var**0.5
total_interval = total + total_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))
print("Confidence interval: ["+ str(round(total_interval[0],2))+ ", "+ str(round(total_interval[1],2))+"]")

# Accuracy of estimation
if p_total > total_interval[0] and p_total < total_interval[1]:
    print("\nValid!")
else:
    print("\nNot valid!")

Population | total: 709408
Sample     | total: 715400
Confidence interval: [650219.62, 780580.38]

Valid!


### 4] Estimador de una proporción

    - Sea y una variable binaria [0, 1], la cual indique que la unidad posee o no una cualidad, se define la proporción de aquellos que tienen una cualidad dentro de la población como:
        prop = sum(y_i) / N
   
    - El análisis estadístico debe realizarse en cualidades que sean -comunes- dentro de la población, donde aproximadamente prop > 0.2
        
    - Los estimadores insesgados son:
        prop = sum(y_i) / n = mean
        V(prop) = (1-n/N) * 1/n * N/(N-1) * prop*(1-prop)
        
    - Los estimadores insesgados para estimar el número de unidades que cuenten con la característica dentro de la población:
        N_prop = N*prop
        V(N_prop) = N^2 * V(prop)

In [420]:
# Analysis is done using "Is_male" variable


# Population parameters
p_prop = sum(df_population["is_male"]) / N
p_N_prop = sum(df_population["is_male"])
print("Population | prop: "+str(round(p_prop,2)) + " N_prop: "+ str(int(p_N_prop)))

# Estimate parameters
prop = df_sample["is_male"].sum() / n
N_prop = N*prop
print("Sample     | prop: "+str(round(prop,2)) + " N_prop: "+ str(int(N_prop)))

# Calculate confidence intervals
prop_var = (1-n/N) * 1/n * N/(N-1) * prop*(1-prop)
prop_std = prop_var**0.5
prop_interval = prop + prop_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))
print("Confidence interval: ["+ str(round(prop_interval[0],2))+ ", "+ str(round(prop_interval[1],2))+"]" + " and ["+ str(int(N*prop_interval[0]))+ ", "+ str(int(N*prop_interval[1]))+"]")

# Accuracy of estimation
if p_prop > prop_interval[0] and p_prop < prop_interval[1]:
    print("\nValid!")
else:
    print("\nNot valid!")

Population | prop: 0.14 N_prop: 137
Sample     | prop: 0.14 N_prop: 140
Confidence interval: [0.06, 0.22] and [59, 220]

Valid!
